# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os
%matplotlib inline

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
city_df_path = "output_data/city-df.csv"


# Read the mouse data and the study results
city_data = pd.read_csv(city_df_path)
city_data


,City,Latitude,Longitude,Humidity,Temperature,Temp Max,Cloudiness,Wind Speed
0,kadykchan,-46.1927,168.8643,99.0,46.47,46.47,99.0,3.51
1,mataura,41.7597,86.1469,13.0,81.01,81.01,100.0,2.57
2,korla,-33.0153,27.9116,36.0,77.94,77.94,0.0,4.61
3,east london,46.6811,-68.0159,88.0,52.41,54.28,90.0,9.22
4,belushya guba,34.6391,-120.4579,82.0,60.96,73.78,1.0,3.44
...,...,...,...,...,...,...,...,...
583,berezna,34.6851,135.8049,96.0,70.52,72.64,100.0,5.59
584,yershov,33.0833,-16.3333,77.0,65.62,70.54,75.0,16.11
585,pasni,65.0083,53.9113,58.0,63.28,63.28,72.0,10.49
586,caibarien,16.1833,-22.9167,85.0,71.20,71.20,41.0,15.17


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [3]:
# Configure gmaps with API key
gmaps.configure(api_key=g_key)


In [4]:
# Store 'Lat' and 'Lng' into  locations 
locations = city_data[["Latitude", "Longitude"]].astype(float)

# Convert Poverty Rate to float and store
# HINT: be sure to handle NaN values
city_data = city_data

humidity = city_data["Humidity"].astype(float)

In [6]:
# Create a poverty Heatmap layer
fig = gmaps.figure()

heat_layer = gmaps.heatmap_layer(locations, weights=humidity, 
                                 dissipating=False, max_intensity=10,
                                 point_radius = 1)

fig.add_layer(heat_layer)
fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [7]:
city_ideal = city_data.loc[(city_data["Temperature"] < 80) & (city_data["Temperature"] > 70) & (city_data["Wind Speed"] < 10) & (city_data["Cloudiness"] == 0)]
city_ideal

,City,Latitude,Longitude,Humidity,Temperature,Temp Max,Cloudiness,Wind Speed
2,korla,-33.0153,27.9116,36.0,77.94,77.94,0.0,4.61
113,hithadhoo,28.0170,-97.5169,89.0,77.07,81.84,0.0,3.76
126,sorvag,-24.5333,32.9833,41.0,78.46,78.46,0.0,7.78
322,beatrice,29.5688,-97.9647,91.0,70.81,74.59,0.0,5.14
324,portland,-8.2578,-49.2647,100.0,71.73,71.73,0.0,3.44
328,roma,48.0056,91.6419,22.0,71.69,71.69,0.0,8.52
528,marsh harbour,38.1460,13.0844,58.0,76.96,81.09,0.0,6.91


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [ ]:
hotels = []

In [ ]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [ ]:
# Add marker layer ontop of heat map


# Display figure
